# SOLAR-NEUTRINO-VISIBLE-DECAYS

https://github.com/mhostert/solar-neutrino-visible-decays

## Import modules

In [1]:
import numpy as np
from scipy import interpolate
import matplotlib
# matplotlib.use('agg')
import matplotlib.pyplot as plt
from matplotlib import rc, rcParams
from matplotlib.pyplot import *
from scipy.stats import chi2
import importlib
import vegas
import gvar as gv

from source import *
from source import flavour_transitions as osc

{'interp': <scipy.interpolate.interpolate.RegularGridInterpolator object at 0x7f69616c4f90>}


## Pick points to sample integrand

In [2]:
##########
# integration evaluations
rates.NEVALwarmup = 1e3
rates.NEVAL = 1e4

## Flux and Decay Parameters

The mixings will be used to rescale the number of events, so pick anything sensible

In [3]:
###########
# NUMU FLUX
fluxfile = "fluxes/b8spectrum.txt"
flux = fluxes.get_exp_flux(fluxfile)

###########
# DECAY MODEL PARAMETERS
params = model.decay_model_params(const.SCALAR)
params.gx		= 1.0
params.Ue4		= np.sqrt(0.01)
params.Umu4		= np.sqrt(0.01)
params.Utau4    = np.sqrt(0)
params.UD4		= np.sqrt(1.0-params.Ue4*params.Ue4-params.Umu4*params.Umu4)
params.m4		= 100e-9 # GeV
params.mBOSON  = 0.5*params.m4 # GeV

###########
# EXPERIMENTS
KAM = exps.kamland_data()
BOR = exps.borexino_data()
SK = exps.superk_data()


# Compute rates for benchmark point at the three experiments

This can be an expensive computation, depending on the desired precision -- we care about tails.

In [4]:
bK, npK, backK, dK = rates.fill_bins(KAM,params,fluxfile,endpoint=16.3)
bB, npB, backB, dB = rates.fill_bins(BOR,params,fluxfile,startpoint=0,endpoint=16.3)
bS, npS, backS, dS = rates.fill_bins(SK,params,fluxfile,endpoint=16.3)


Filling the bins in kamland
Filling the bins in borexino
Filling the bins in SUPERK_IV


In [5]:

bK = bK[:-1] + (bK[1:] - bK[:-1])/2
bB = bB[:-1] + (bB[1:] - bB[:-1])/2
bS = bS[:-1] + (bS[1:] - bS[:-1])/2

## Fill arrays with rescale number of events depending on Umu4 and Ue4
importlib.reload(const)
importlib.reload(osc)


In [6]:
importlib.reload(const)
importlib.reload(osc)

##############
# Bin uncorrelated normalization errors
err_flux = 0.1
err_backK = 0.1
err_backB = 0.1
err_backS = 0.1
############# 
# 2D grid in Umu4 and Ue4 space
NPOINTS = 33
UE4SQR =np.logspace(-4,-1,NPOINTS)
UMU4SQR =np.logspace(-4,-1.1,NPOINTS)
LK = np.zeros((NPOINTS,NPOINTS))
LB = np.zeros((NPOINTS,NPOINTS))
LS = np.zeros((NPOINTS,NPOINTS))

#############
# number of degrees of freedom
dofK = np.size(dK)-2
dofB = np.size(dB)-2
dofS = np.size(dS)-2



# Initial scaling factor
old_factorK = params.Ue4**2*(osc.Pse(bK, params.Ue4*params.Ue4, params.Umu4*params.Umu4))
old_factorB = params.Ue4**2*(osc.Pse(bB, params.Ue4*params.Ue4, params.Umu4*params.Umu4))
old_factorS = params.Ue4**2*(osc.Pse(bS, params.Ue4*params.Ue4, params.Umu4*params.Umu4))

{'interp': <scipy.interpolate.interpolate.RegularGridInterpolator object at 0x7f695f4e1450>}


# Minimize likelihood over nuisance parameters for every point

In [7]:
importlib.reload(stats)

for i in range(np.size(UE4SQR)):
    for j in range(np.size(UMU4SQR)):
        new_factorK = UE4SQR[i]*osc.Pse(bK, UE4SQR[i], UMU4SQR[j])
        new_factorB = UE4SQR[i]*osc.Pse(bB, UE4SQR[i], UMU4SQR[j])
        new_factorS = UE4SQR[i]*osc.Pse(bS, UE4SQR[i], UMU4SQR[j])
        np_newK = new_factorK/old_factorK*npK
        np_newB = new_factorB/old_factorB*npB
        np_newS = new_factorS/old_factorS*npS
        # print new_factorK,new_factorB
        LK[j,i] = stats.chi2_binned_rate(np_newK, backK, dK, [err_flux,err_backK])#np.sum(np_newK)
        LB[j,i] = stats.chi2_binned_rate(np_newB, backB, dB, [err_flux,err_backB])#np.sum(np_newB)
        LS[j,i] = stats.chi2_binned_rate(np_newS, backS, dS, [err_flux,err_backS])#np.sum(np_newS)
#         LK[j,i] = stats.chi2_total_rate(np_newK, backK, dK, [err_flux,err_backK])
#         LB[j,i] = stats.chi2_total_rate(np_newB, backB, dB, [err_flux,err_backB])
#         LS[j,i] = stats.chi2_total_rate(np_newS, backS, dS, [err_flux,err_backS])
print(np.min(LK), dofK)
LK = LK - np.min(LK)
print(np.min(LB), dofB)
LB = LB - np.min(LB)
print(np.min(LS), dofS)
LS = LS - np.min(LS)

5.20510851924564 6
19.739876035591934 12
3.3573693658744004 5


# Plot resulting limits

### Setup and plot preference regions from https://arxiv.org/abs/1911.01427 

In [40]:
################################################################
# PLOTTING THE LIMITS
################################################################
fsize=11
rc('text', usetex=True)
rcparams={'axes.labelsize':fsize,'xtick.labelsize':fsize,'ytick.labelsize':fsize,\
				'figure.figsize':(1*3.7,1.4*2.3617)	}
rc('font',**{'family':'serif', 'serif': ['computer modern roman']})
matplotlib.rcParams['hatch.linewidth'] = 0.1  # previous pdf hatch linewidth
rcParams.update(rcparams)
axes_form  = [0.185,0.15,0.775,0.76]
fig = plt.figure()
ax = fig.add_axes(axes_form)

############
# GET THE FIT REGIONS FROM DENTLER ET AL
DentlerPath='digitized/Dentler_et_al/0.5_100/'

SB_COLOR = 'lightgrey'
MB_ue_b,MB_umu_b = np.genfromtxt(DentlerPath+'bottom_MiniBooNE.dat',unpack=True)
MB_ue_t,MB_umu_t = np.genfromtxt(DentlerPath+'top_MiniBooNE.dat',unpack=True)
MB_ue_f=np.logspace( np.log10(np.min([MB_ue_b])), np.log10(np.max([MB_ue_b])), 100)
MB_umu_b_f = np.interp(MB_ue_f,MB_ue_b,MB_umu_b)
MB_umu_t_f = np.interp(MB_ue_f,MB_ue_t,MB_umu_t)
ax.fill_between(MB_ue_f,MB_umu_b_f,MB_umu_t_f,facecolor=SB_COLOR,alpha=0.5,lw=0)
ax.fill_between(MB_ue_f,MB_umu_b_f,MB_umu_t_f,edgecolor='black',facecolor='None',lw=0.6)

y,x = np.genfromtxt(DentlerPath+'right_LSND.dat',unpack=True)
yl,xl = np.genfromtxt(DentlerPath+'left_LSND.dat',unpack=True)
x_f=np.logspace( np.log10(np.min([x])), np.log10(np.max([x])), 100)
y_f = np.interp(x_f,x,y)
yl_f = np.interp(x_f,xl,yl)
ax.fill_betweenx(x_f,y_f,yl_f,facecolor=SB_COLOR,alpha=0.5,lw=0)
ax.fill_betweenx(x_f,y_f,yl_f,edgecolor='black',facecolor='None',lw=0.6)

x,y = np.genfromtxt(DentlerPath+'bottom_combined.dat',unpack=True)
xl,yl = np.genfromtxt(DentlerPath+'top_combined.dat',unpack=True)
x_f=np.logspace( np.log10(np.min([x])), np.log10(np.max([x])), 100)
y_f = np.interp(x_f,x,y)
yl_f = np.interp(x_f,xl,yl)
ax.fill_between(x_f,y_f,yl_f,facecolor='None',edgecolor='yellow',
                hatch='/////////////////////////////////////////',alpha=0.7,lw=0,zorder=10)
ax.fill_between(x_f,y_f,yl_f,edgecolor='black',facecolor='None',lw=0.6,zorder=10)


x,y = np.genfromtxt(DentlerPath+'KARMEN.dat',unpack=True)
ax.plot(x,y,lw=0.8,color='gray',dashes=(1,1))
x,y = np.genfromtxt(DentlerPath+'OPERA.dat',unpack=True)
ax.plot(x,y,lw=0.8,color='gray',dashes=(1,1))


X,Y = np.meshgrid(UE4SQR,UMU4SQR)
# ax.contourf(X,Y,LK, [chi2.ppf(0.90, dofK),1e100], colors=['black'],alpha=0.1, linewidths=[0.1])
# ax.contour(X,Y,L, 20, color='black')

# ax.contourf(X,Y,LB, [chi2.ppf(0.90, dofB),1e100], colors=['black'],alpha=0.1, linewidths=[0.1])

# Z = LB
# pcm = ax.pcolor(X, Y, Z,
#                    norm=colors.LogNorm(vmin=Z.min(), vmax=Z.max()),
#                    cmap='PuBu_r')# ax.contour(X,Y,L, 20, color='black')
# fig.colorbar(pcm, ax=ax, extend='max')

c1=ax.contour(X,Y,LK, [chi2.ppf(0.99, dofK)], linestyles=['--'],colors=['magenta'],linewidths=[1.0],label=r'KamLAND')
c2=ax.contour(X,Y,LS, [chi2.ppf(0.99, dofS)], linestyles=['--'],colors=['dodgerblue'],linewidths=[1.0], ls='--',label=r'SuperK-IV')
c3=ax.contour(X,Y,LB, [chi2.ppf(0.99, dofB)], linestyles=['--'],colors=['indigo'],linewidths=[1.0],label=r'Borexino')
c1.collections[0].set_dashes([(0, (2.0, 0))])
c2.collections[0].set_dashes([(0, (7.0, 1.0))])
c3.collections[0].set_dashes([(0, (2.0, 1.0))])
h1,_ = c1.legend_elements()
h2,_ = c2.legend_elements()
h3,_ = c3.legend_elements()

ax.legend([h1[0], h3[0],h2[0]], ['KamLAND', 'Borexino','SuperK-IV'],loc='uppwer right', frameon=False,fontsize=9)





# ax.clear()
# ax.set_xscale('log')
# ax.set_yscale('log')
##############
# STYLE
if params.model == const.VECTOR:
	boson_string = r'$m_{Z^\prime}$'
	boson_file = 'vector'
elif params.model == const.SCALAR:
	boson_string = r'$m_\phi$'
	boson_file = 'scalar'

# RESCALE=1.27*4
# RESCALEY = 0.5
# # ax.annotate(r'', fontsize=fsize, xy=(RESCALE*4.4e-3,RESCALEY*6e-4), xytext=(RESCALE*2.65e-3,RESCALEY*6e-4),color='blue',
#             arrowprops=dict(arrowstyle="-|>", mutation_scale=5, color='red', lw = 0.5),
#             )
# ax.annotate(r'', fontsize=fsize, xy=(RESCALE*5.5e-3,RESCALEY*7e-4), xytext=(RESCALE*3.4e-3,RESCALEY*7e-4),color='blue',
#             arrowprops=dict(arrowstyle="-|>", mutation_scale=5, color='blue', lw = 0.5),
#             )
# ax.annotate(r'', fontsize=fsize, xy=(RESCALE*6.4e-3,RESCALEY*8.1e-4), xytext=(RESCALE*4.05e-3,RESCALEY*8.1e-4),color='blue',
#             arrowprops=dict(arrowstyle="-|>", mutation_scale=5, color='green', lw = 0.5),
#             )
# ax.annotate(r'KamLAND $90\%$ C.L.', fontsize=0.8*fsize, xy=(0.45,0.17), xytext=(0.3,0.18),xycoords='axes fraction', color='blue')
# ax.annotate(r'\noindent99\% C.L.\\excluded', fontsize=0.8*fsize, xy=(0.7,0.2), xytext=(0.73,0.02),xycoords='axes fraction', color='black')

ax.set_title(r'$m_4 \Gamma_4 = 1$ eV$^2$,\, $m_4 = %.0f$ eV,\, '%(params.m4*1e9)+boson_string+r'$/m_4 = %.1f$'%(params.mBOSON/params.m4), fontsize=9)
ax.annotate(r'MiniBooNE',xy=(0.4,0.031),xycoords='axes fraction',color='black',fontsize=9,rotation=-2)
ax.annotate(r'LSND',xy=(0.5,0.53),xycoords='axes fraction',color='black',fontsize=8,rotation=0)
ax.annotate(r'\noindent All w/o \\LSND',xy=(0.21,0.05),xycoords='axes fraction',color='black',fontsize=8,rotation=0,zorder=10)

ax.annotate(r'\noindent 99\% C.L. \\\noindent excluded', 
            fontsize=0.9*fsize, xy=(1.2,0.2), xytext=(0.34,0.75),xycoords='axes fraction', color='magenta')

ax.annotate(r'', fontsize=fsize, xy=(0.045,0.0175), xytext=(0.021,0.0175), xycoords='data' ,zorder=1000, color='magenta',
arrowprops=dict(arrowstyle="-|>", mutation_scale=5, color='magenta', lw = 0.8))

ax.fill_between(np.linspace(9e-3,0.1,100),0.7e-3*np.ones(100),-0.05e-3*np.ones(100),
                lw=0.0,fc='None',ec='darkgreen',
                hatch='//////////////////////////////////////',zorder=2)
ax.annotate(r'Reactors preferred',xy=(0.066,0.13e-3),
                xycoords='data',color='white',fontsize=7.5,rotation=0)
# ax.annotate(r'', fontsize=fsize, xy=(0.022,0.3e-3), xytext=(9e-3,0.3e-3), xycoords='data' ,zorder=1000, color='magenta',
# arrowprops=dict(arrowstyle="|-|", mutation_scale=2, color='black', lw = 0.8))
# ax.annotate(r'', fontsize=fsize, xy=(0.04,0.3e-3), xytext=(0.0355,0.3e-3), xycoords='data' ,zorder=1000, color='magenta',
# arrowprops=dict(arrowstyle="-|>", mutation_scale=3, color='black', lw = 0.8))
ax.vlines(0.031,-1,1,lw=1.0, ls=':',color='black',zorder=-1)
ax.annotate(r'OPERA',xy=(0.08,0.0076), 
            xycoords='data',color='grey',fontsize=8,rotation=0)
ax.annotate(r'KARMEN',xy=(0.042,0.0030), 
            xycoords='data',color='grey',fontsize=8,rotation=-60)
ax.annotate(r'$\beta$-decay',xy=(0.032,0.0043), 
            xycoords='data',color='grey',fontsize=8,rotation=-90)

# ax.annotate(r'Borexino',xy=(0.55,0.35),xycoords='axes fraction',fontsize=14)
ax.set_xlim(0,0.1)
ax.set_ylim(0,0.02)
ax.set_yticks([0,0.005,0.01,0.015,0.02])
ax.set_xlabel(r'$|U_{e 4}|^2$')
ax.set_ylabel(r'$|U_{\mu 4}|^2$')
fig.savefig('plots/limits_MN_%.0f_MB_%.0f.pdf'%(params.m4*1e9,params.mBOSON*1e9),rasterized=True)
fig.show()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:64: UserWarning: The following kwargs were not used by contour: 'label'
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:65: UserWarning: The following kwargs were not used by contour: 'ls', 'label'
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:66: UserWarning: The following kwargs were not used by contour: 'label'
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:74: MatplotlibDeprecationWarning: Unrecognized location 'uppwer right'. Falling back on 'best'; valid locations are
	best
	upper right
	upper left
	lower left
	lower right
	rig